In [1]:
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegressionCV
import sklearn.metrics as metrics
from sklearn.metrics import r2_score
from sklearn.preprocessing import PolynomialFeatures
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score
import sklearn.discriminant_analysis as da
import sklearn.neighbors as knn
from sklearn.model_selection import KFold
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_curve, auc
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import RidgeCV
from sklearn.linear_model import LassoCV
from statsmodels.api import OLS
from statsmodels.api import add_constant
import statsmodels.api as sm
import datetime

#import pydotplus
#import io
from sklearn.tree import export_graphviz
from IPython.display import Image
from IPython.display import display
%matplotlib inline
from matplotlib import pyplot
default_dims = (13, 10)
import seaborn.apionly as sns #sets up styles and gives us more plotting options
sns.set_style("whitegrid")
sns.set_context("poster")
sns.reset_orig()

/anaconda/lib/python3.6/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


To-do:
* Find profit & loss for our positions, graph it over time against the S&P
* Find current positions
* Do a factor analysis: Run a regression of P&L against factor data

In [3]:
filenames = ["Transaction Data/transactions.csv"]
filenames = filenames + ["Transaction Data/transactions ({}).csv".format(i) for i in range(1,7)]

dfs = []
for filename in filenames:
    dfs.append(pd.read_csv(filename, index_col = None))

# Adds in last dataframe at the end to keep them in order
df = pd.concat(dfs)
print(np.shape(df))
df.head()

(835, 13)


,DATE,TRANSACTION ID,DESCRIPTION,QUANTITY,SYMBOL,PRICE,COMMISSION,AMOUNT,NET CASH BALANCE,REG FEE,SHORT-TERM RDM FEE,FUND REDEMPTION FEE,DEFERRED SALES CHARGE
0,09/29/2017,1.760966e+10,MONEY MARKET INTEREST (MMDA1),0.37,MMDA1,NaN,NaN,0.00,---,NaN,NaN,NaN,NaN
1,09/13/2017,1.750578e+10,MONEY MARKET PURCHASE,NaN,NaN,NaN,NaN,-4943.94,---,NaN,NaN,NaN,NaN
2,09/13/2017,1.750578e+10,MONEY MARKET PURCHASE (MMDA1),4943.94,MMDA1,NaN,NaN,0.00,---,NaN,NaN,NaN,NaN
3,09/11/2017,1.749263e+10,Sold 450 LIND @ 11,450.00,LIND,11.0,5.95,4943.94,---,0.11,NaN,NaN,NaN
4,08/31/2017,1.743493e+10,MONEY MARKET INTEREST (MMDA1),0.33,MMDA1,NaN,NaN,0.00,---,NaN,NaN,NaN,NaN


In [26]:
df.tail()

,DATE,TRANSACTION ID,DESCRIPTION,QUANTITY,SYMBOL,PRICE,COMMISSION,AMOUNT,NET CASH BALANCE,REG FEE,SHORT-TERM RDM FEE,FUND REDEMPTION FEE,DEFERRED SALES CHARGE
229,11/03/2010,6.508642e+09,MONEY MARKET REDEMPTION (MMDA10),122.64,MMDA10,NaN,NaN,0.00,---,NaN,NaN,NaN,NaN
230,11/03/2010,6.508642e+09,MONEY MARKET REDEMPTION,NaN,NaN,NaN,NaN,122.64,---,NaN,NaN,NaN,NaN
231,11/02/2010,6.502567e+09,MARK TO THE MARKET,NaN,NaN,NaN,NaN,122.64,---,NaN,NaN,NaN,NaN
232,11/02/2010,6.502567e+09,MARK TO THE MARKET,NaN,NaN,NaN,NaN,-122.64,---,NaN,NaN,NaN,NaN
233,***END OF FILE***,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [27]:
df_old = df.copy()

In [28]:
df = df[~df['PRICE'].isnull()]
df.index = range(len(df))

In [29]:
df[df['PRICE'].notnull()]
stocks = list(set(df['SYMBOL']))

In [30]:
df['NET_CHANGE'] = [df['QUANTITY'][i] if 'Bought' in str(df['DESCRIPTION'][i]) else -df['QUANTITY'][i] if 'Sold' in str(df['DESCRIPTION'][i]) else 0 for i in range(len(df))] 

In [31]:
'Bought' in list(df['DESCRIPTION'])[13]

True

In [32]:
df[df['SYMBOL'] == 'AAPL']

,DATE,TRANSACTION ID,DESCRIPTION,QUANTITY,SYMBOL,PRICE,COMMISSION,AMOUNT,NET CASH BALANCE,REG FEE,SHORT-TERM RDM FEE,FUND REDEMPTION FEE,DEFERRED SALES CHARGE,NET_CHANGE
73,10/31/2012,9.224839e+09,Sold 7 AAPL @ 594.49,7.0,AAPL,594.49,9.99,4151.34,---,0.1,NaN,NaN,NaN,-7.0
122,04/25/2011,7.114304e+09,Bought 7 AAPL @ 351.69,7.0,AAPL,351.69,9.99,-2471.82,---,NaN,NaN,NaN,NaN,7.0


In [33]:
grouped = df.groupby(['SYMBOL']).sum()

In [34]:
grouped.index

Index(['AAPL', 'AFG', 'AGG', 'AMRK', 'AMZKF', 'ARO', 'AXANF', 'AXLE', 'BAMM',
       'BBY', 'BETM', 'CCX', 'CDCO', 'CEG', 'CMG', 'COH', 'CPC', 'CTO', 'CWGL',
       'DANG', 'DV', 'DX', 'GLGI', 'GRVY', 'INFU', 'INSW', 'IOC', 'LIND',
       'LMAT', 'LPHI', 'LVB', 'LYB', 'MRK', 'NAUH', 'NILSY', 'NLY', 'NOTEX',
       'NROM', 'NUSMF', 'OMEX', 'ORM', 'PBR', 'PFE', 'PG', 'PKTEF', 'PRSS',
       'PRXI', 'PTNT', 'PW', 'RGS', 'RHDGF', 'RIO', 'SHFK', 'SONE', 'SPA',
       'SPGZ', 'STRP', 'STTYF', 'SYNA', 'TEVA', 'TNH', 'UPL', 'USAK', 'WBMD',
       'WDC', 'XIN'],
      dtype='object', name='SYMBOL')

In [35]:
grouped.head()

,TRANSACTION ID,QUANTITY,PRICE,COMMISSION,AMOUNT,REG FEE,SHORT-TERM RDM FEE,FUND REDEMPTION FEE,DEFERRED SALES CHARGE,NET_CHANGE
SYMBOL,,,,,,,,,,
AAPL,1.633914e+10,14.0,946.1800,19.98,1679.52,0.10,NaN,NaN,NaN,0.0
AFG,1.387654e+10,86.0,66.6800,19.98,194.99,0.03,NaN,NaN,NaN,0.0
AGG,9.598242e+09,38.0,110.0201,0.00,4180.66,0.10,NaN,NaN,NaN,-38.0
AMRK,1.248004e+10,243.0,9.8501,7.00,2386.51,0.06,NaN,NaN,NaN,-243.0
AMZKF,3.690929e+10,4000.0,5.6760,16.99,-922.77,0.08,NaN,NaN,NaN,0.0


In [36]:
grouped['NET_EXPOSURE'] = [max(0,grouped['NET_CHANGE'][i] * grouped['PRICE'][i]) for i in range(len(grouped['NET_CHANGE']))]

In [37]:
long_positions = grouped[grouped['NET_CHANGE'] > 0]

In [38]:
long_positions.index

Index(['AXLE', 'INSW', 'IOC', 'NILSY', 'NUSMF', 'RHDGF', 'SPGZ', 'STTYF',
       'USAK'],
      dtype='object', name='SYMBOL')

In [39]:
short_positions = grouped[grouped['NET_CHANGE'] < 0]

In [40]:
short_positions.index

Index(['AGG', 'AMRK', 'CDCO', 'CEG', 'CPC', 'LPHI', 'NLY', 'PFE', 'PTNT',
       'SONE', 'WDC'],
      dtype='object', name='SYMBOL')

In [41]:
grouped.loc[long_positions.index]

,TRANSACTION ID,QUANTITY,PRICE,COMMISSION,AMOUNT,REG FEE,SHORT-TERM RDM FEE,FUND REDEMPTION FEE,DEFERRED SALES CHARGE,NET_CHANGE,NET_EXPOSURE
SYMBOL,,,,,,,,,,,
AXLE,1.318974e+10,2000.0,0.4650,7.00,-937.00,NaN,NaN,NaN,NaN,2000.0,930.000
INSW,7.771543e+09,200.0,7.9190,9.99,-1593.79,NaN,NaN,NaN,NaN,200.0,1583.800
IOC,6.552417e+09,14.0,79.1400,9.99,-1117.95,NaN,NaN,NaN,NaN,14.0,1107.960
NILSY,7.672488e+09,118.0,20.9200,9.99,-2478.55,NaN,NaN,NaN,NaN,118.0,2468.560
NUSMF,1.684582e+10,8000.0,0.1881,5.95,-1510.75,NaN,NaN,NaN,NaN,8000.0,1504.800
RHDGF,1.654202e+10,110.0,18.5500,5.95,-2046.45,NaN,NaN,NaN,NaN,110.0,2040.500
SPGZ,1.123502e+10,975.0,2.9600,9.99,-2895.99,NaN,NaN,NaN,NaN,975.0,2886.000
STTYF,1.870512e+10,4860.0,1.8585,19.98,-1282.91,0.02,NaN,NaN,NaN,3978.0,7393.113
USAK,2.707981e+10,180.0,44.4890,14.00,-3951.90,NaN,NaN,NaN,NaN,180.0,8008.020


In [42]:
grouped['NET_EXPOSURE'] = [max(0,grouped['NET_CHANGE'][i] * grouped['PRICE'][i]) for i in range(len(grouped['NET_CHANGE']))]

In [43]:
grouped.loc[long_positions.index]

,TRANSACTION ID,QUANTITY,PRICE,COMMISSION,AMOUNT,REG FEE,SHORT-TERM RDM FEE,FUND REDEMPTION FEE,DEFERRED SALES CHARGE,NET_CHANGE,NET_EXPOSURE
SYMBOL,,,,,,,,,,,
AXLE,1.318974e+10,2000.0,0.4650,7.00,-937.00,NaN,NaN,NaN,NaN,2000.0,930.000
INSW,7.771543e+09,200.0,7.9190,9.99,-1593.79,NaN,NaN,NaN,NaN,200.0,1583.800
IOC,6.552417e+09,14.0,79.1400,9.99,-1117.95,NaN,NaN,NaN,NaN,14.0,1107.960
NILSY,7.672488e+09,118.0,20.9200,9.99,-2478.55,NaN,NaN,NaN,NaN,118.0,2468.560
NUSMF,1.684582e+10,8000.0,0.1881,5.95,-1510.75,NaN,NaN,NaN,NaN,8000.0,1504.800
RHDGF,1.654202e+10,110.0,18.5500,5.95,-2046.45,NaN,NaN,NaN,NaN,110.0,2040.500
SPGZ,1.123502e+10,975.0,2.9600,9.99,-2895.99,NaN,NaN,NaN,NaN,975.0,2886.000
STTYF,1.870512e+10,4860.0,1.8585,19.98,-1282.91,0.02,NaN,NaN,NaN,3978.0,7393.113
USAK,2.707981e+10,180.0,44.4890,14.00,-3951.90,NaN,NaN,NaN,NaN,180.0,8008.020


In [44]:
np.sum(grouped['NET_EXPOSURE'])

27922.753

In [45]:
factors = pd.read_csv('F-F_Research_Data_5_Factors_2x3_daily.CSV', index_col = 'Date')

In [46]:
stocks_held = ['USAK', 'RHDGF', 'DXLG', 'NUSMF', 'LEE', 'AXLE']

In [48]:
quantities = [180, 110, 1000, 8000, 425, 2000]